This Notebook I will test how to implement a function that will take a csv and create a partitioned parquet file

Testing partition locally to see if it works with pyspark

cleaning my csv file with Csv class but changing the class so it doesnt convert to parquet. This is because its better to write the partitions directly to a parquet file.

In [3]:
import pandas as pd
import os

In [13]:
class CsvCleaner:
    @staticmethod
    def timestamp_clean(df, col_name):
        # convert the column to numeric with any errors(for example strings or letter) to NaN
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
        
        df.dropna(subset=[col_name], inplace=True)

        # Calculate the initial number of rows
        initial_rows = df.shape[0]
        
        #Filter out rows with "Timestamp" values not containing 10 digits
        df = df[df[col_name].apply(lambda x: len(str(int(x))) == 10)]

        #calculate how many rows removed
        rows_removed = initial_rows - df.shape[0]

        # Convert the Unix timestamp to datetime with seconds
        df[col_name] = pd.to_datetime(df[col_name], unit="s")

        # Sort the DataFrame by the timestamp column
        df = df.sort_values(by=col_name)

        return df, rows_removed
    
    @staticmethod
    def clean_columns(df, col_name, low, high):
    
        # Convert column to numeric, making errors to Nan instead
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")

        # Calculate the initial number of rows
        initial_rows = df.shape[0]

        df.loc[~df[col_name].between(low, high), col_name] = float('nan')

        # Calculate the number of rows removed
        rows_removed = initial_rows - df.shape[0]

        #df[col_name] = df[col_name].interpolate()

        return df, rows_removed
    
    @staticmethod
    def clean_file(csv_file):
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file)
        total_rows_removed = 0

        # Clean the DataFrame
        for col in df.columns:
            if "Timestamp" in col:
                df, rows_removed = CsvCleaner.timestamp_clean(df, col)
                total_rows_removed += rows_removed
            
            if "speed_over_ground" in col:
                low = 0
                high = 100
                df, rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed
            
            if "Longitude" in col:
                low = -180
                high = 180
                df, rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed

            if "Latitude" in col:
                low = -90
                high = 90
                df, rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed

            if "engine_fuel_rate" in col:
                low = 0
                high = 100
                df,rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed

        output_dir = os.path.dirname(csv_file)
        file_name = os.path.basename(csv_file)
        file_name_without_extension, extension = os.path.splitext(file_name)
        cleaned_csv_file = os.path.join(output_dir, file_name_without_extension + "_cleaned.csv")

        df.to_csv(cleaned_csv_file, index=False)

        print(f"Total rows removed: {total_rows_removed}")

        return cleaned_csv_file

In [14]:
vessel_1 = "../src/Data/vessel1_dummy_boat_data.csv"
vessel_2 = "../src/Data/vessel2_dummy_boat_data.csv"

In [17]:
# Create an instance of CsvCleaner
cleaner = CsvCleaner()

# Clean the CSV file
vessel_1_csv_file_path = cleaner.clean_file(vessel_1)
vessel_2_csv_file_path = cleaner.clean_file(vessel_2)

Total rows removed: 5
Total rows removed: 5


/tmp/ipykernel_1319/2019830326.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], unit="s")
/tmp/ipykernel_1319/2019830326.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], unit="s")


Now that I have cleaned the csv files I will now try and partition and create parquet files locally to see if the process works. Then I will try and connect the process with my personal AWS and with my lambda function.

In [1]:
from pyspark import SparkSession
from pyspark.sql.functions import input_file_name, year, month, dayofmonth, regexp_extract

# Initialize a Spark Session
spark = SparkSession.builder \
    app.Name("LocalParquetConversion") \
    .getOrCreate()

# Path to local sample dataset
source_path = "file:///../src/Data/cleaned_csv_files"

# Read data from local files
source_df = spark.read.csv(source_path, header=True)

# Extract vessel name from file name
source_df = source_df.withColumn("file_name", input_file_name())
source_df = source_df.withColumn("vessel", regexp_extract("filename", "([^\/]+)(?=\.[\w]+$)", 1))



ImportError: cannot import name 'SparkSession' from 'pyspark' (/home/scglass/Github/LIA/parquet_conversion/venv/lib/python3.11/site-packages/pyspark/__init__.py)